In [17]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/sambarati/Documents/GitHub/mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [18]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

In [19]:
MULTIPLE_SENSORS = True

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [ ]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# Try to get API key from environment variable first (GitHub Actions), then Hopsworks secrets
AQICN_API_KEY = os.getenv("AQICN_API_KEY")
if AQICN_API_KEY is None:
    api_key_secret = secrets.get_secret("AQICN_API_KEY")
    if api_key_secret is not None:
        AQICN_API_KEY = api_key_secret.value
    else:
        raise ValueError("AQICN_API_KEY not found in environment variables or Hopsworks secrets")

today = datetime.date.today()

2025-11-18 00:10:21,913 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-18 00:10:21,915 INFO: Initializing external client
2025-11-18 00:10:21,915 INFO: Base URL: https://c.app.hopsworks.ai:443
Connection closed.
2025-11-18 00:10:21,915 INFO: Initializing external client
2025-11-18 00:10:21,915 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 00:10:23,525 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1267871

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1267871


In [ ]:
if not MULTIPLE_SENSORS:
    # Single sensor mode - try environment variables first, then Hopsworks secrets
    location_str = os.getenv("SENSOR_LOCATION_JSON")
    if location_str is None:
        location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
    
    location = json.loads(location_str)
    country=location['country']
    city=location['city']
    street=location['street']
    aqicn_url=location['aqicn_url']
    latitude=location['latitude']
    longitude=location['longitude']
    location_str
else:
    # Multiple sensors mode - try environment variables first, then Hopsworks secrets
    paris_location_str = os.getenv("SENSOR_LOCATION_JSON")
    if paris_location_str is None:
        paris_location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
    paris_location = json.loads(paris_location_str)
    
    birmingham_sensors_str = os.getenv("BIRMINGHAM_SENSOR_LOCATIONS")
    if birmingham_sensors_str is None:
        # Try old name for backwards compatibility
        birmingham_sensors_str = secrets.get_secret("BIRMINGHAM_SENSORS_JSON")
        if birmingham_sensors_str is None:
            birmingham_sensors_str = secrets.get_secret("BIRMINGHAM_SENSOR_LOCATIONS").value
        else:
            birmingham_sensors_str = birmingham_sensors_str.value
    
    birmingham_sensors = json.loads(birmingham_sensors_str)
    
    all_sensors = [paris_location] + birmingham_sensors

In [23]:
all_sensors

[{'country': 'France',
  'city': 'Paris',
  'street': 'Boulevard Peripherique Est',
  'aqicn_url': 'https://api.waqi.info/feed/@3088/',
  'latitude': 48.86,
  'longitude': 2.32},
 {'country': 'United Kingdom',
  'city': 'Birmingham',
  'street': 'Birmingham A4540 Roadside',
  'aqicn_url': 'https://api.waqi.info/feed/@10101/',
  'latitude': '52.476145',
  'longitude': '-1.874978',
  'csv_file': 'birmingham-a4540-roadside-air-quality.csv'},
 {'country': 'United Kingdom',
  'city': 'Birmingham',
  'street': 'Birmingham Ladywood',
  'aqicn_url': 'https://api.waqi.info/feed/@11652/',
  'latitude': '52.481346',
  'longitude': '-1.918235',
  'csv_file': 'birmingham-ladywood-air-quality.csv'},
 {'country': 'United Kingdom',
  'city': 'Birmingham',
  'street': 'Coventry Allesley',
  'aqicn_url': 'https://api.waqi.info/feed/@8913/',
  'latitude': '52.411628',
  'longitude': '-1.560189',
  'csv_file': 'coventry-allesley-air-quality.csv'}]

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [24]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=2,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [25]:
import requests
import pandas as pd

if not MULTIPLE_SENSORS:  
    aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
    aq_today_df
else:
    todays_values = []
    for sensor in all_sensors:
        sensor_df = util.get_pm25(sensor['aqicn_url'], sensor['country'], sensor['city'], sensor['street'], today, AQICN_API_KEY)
        todays_values.append(sensor_df)
    aq_today_df = pd.concat(todays_values, ignore_index=True)
    aq_today_df

In [26]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     4 non-null      float32       
 1   country  4 non-null      object        
 2   city     4 non-null      object        
 3   street   4 non-null      object        
 4   date     4 non-null      datetime64[ns]
 5   url      4 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 308.0+ bytes


## <span style='color:#ff5f27'> 🔄 Calculate Lagged Features</span>

To create the lagged features (pm_25_1_day_lag, pm_25_2_day_lag, pm_25_3_day_lag), we need to fetch the last 3 days of PM2.5 values from the feature group.

In [27]:
if not MULTIPLE_SENSORS:
    # Fetch the last 3 days of PM2.5 data to calculate lagged features
    three_days_ago = today - datetime.timedelta(days=3)

    # Read historical data from the feature group (version 2) - FILTER BY CITY AND STREET
    historical_aq_df = air_quality_fg.filter(
        (air_quality_fg.date >= three_days_ago) & 
        (air_quality_fg.city == city) & 
        (air_quality_fg.street == street)
    ).read()

    # Sort by date to ensure proper ordering
    historical_aq_df = historical_aq_df.sort_values(by='date')

    # Get the last 3 PM2.5 values (most recent first)
    if len(historical_aq_df) >= 3:
        pm25_values = historical_aq_df['pm25'].tail(3).values
        aq_today_df['pm_25_1_day_lag'] = pm25_values[-1]  # Yesterday
        aq_today_df['pm_25_2_day_lag'] = pm25_values[-2]  # 2 days ago
        aq_today_df['pm_25_3_day_lag'] = pm25_values[-3]  # 3 days ago
    else:
        print(f"Warning: Not enough historical data. Found {len(historical_aq_df)} days, need 3.")
        # Set to None or skip insertion if not enough data
        aq_today_df['pm_25_1_day_lag'] = None
        aq_today_df['pm_25_2_day_lag'] = None
        aq_today_df['pm_25_3_day_lag'] = None

    print(f"Lagged features calculated:")
    print(f"  1-day lag (yesterday): {aq_today_df['pm_25_1_day_lag'].values[0]}")
    print(f"  2-day lag: {aq_today_df['pm_25_2_day_lag'].values[0]}")
    print(f"  3-day lag: {aq_today_df['pm_25_3_day_lag'].values[0]}")

    aq_today_df

In [29]:
if MULTIPLE_SENSORS:
    three_days_ago = today - datetime.timedelta(days=3)
    sensor_dfs = []
    
    for i, sensor in enumerate(all_sensors):
        sensor_city = sensor['city']
        sensor_street = sensor['street']
        
        historical_aq_df = air_quality_fg.filter(
            (air_quality_fg.date >= three_days_ago) & 
            (air_quality_fg.city == sensor_city) & 
            (air_quality_fg.street == sensor_street)
        ).read()
        
        historical_aq_df = historical_aq_df.sort_values(by='date')
        
        sensor_today = aq_today_df.iloc[[i]].copy()
        
        if len(historical_aq_df) >= 3:
            pm25_values = historical_aq_df['pm25'].tail(3).values
            sensor_today['pm_25_1_day_lag'] = pm25_values[-1]
            sensor_today['pm_25_2_day_lag'] = pm25_values[-2]
            sensor_today['pm_25_3_day_lag'] = pm25_values[-3]
        else:
            print(f"Warning: Not enough historical data for {sensor_city}/{sensor_street}. Found {len(historical_aq_df)} days, need 3.")
            sensor_today['pm_25_1_day_lag'] = None
            sensor_today['pm_25_2_day_lag'] = None
            sensor_today['pm_25_3_day_lag'] = None
        
        sensor_dfs.append(sensor_today)
    
    aq_today_df = pd.concat(sensor_dfs, ignore_index=True)
    aq_today_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.62s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.62s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.58s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.58s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.61s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.80s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.80s) 


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [30]:
if not MULTIPLE_SENSORS:
    hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
    hourly_df = hourly_df.set_index('date')
    daily_df = hourly_df.between_time('11:59', '12:01')
    daily_df = daily_df.reset_index()
    daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
    daily_df['date'] = pd.to_datetime(daily_df['date'])
    daily_df['city'] = city
    daily_df
else:
    unique_cities = {}
    for sensor in all_sensors:
        sensor_city = sensor['city']
        if sensor_city not in unique_cities:
            unique_cities[sensor_city] = {
                'latitude': sensor['latitude'],
                'longitude': sensor['longitude']
            }
    
    weather_dfs = []
    for city_name, coords in unique_cities.items():
        hourly_df = util.get_hourly_weather_forecast(city_name, coords['latitude'], coords['longitude'])
        hourly_df = hourly_df.set_index('date')
        daily_city_df = hourly_df.between_time('11:59', '12:01')
        daily_city_df = daily_city_df.reset_index()
        daily_city_df['date'] = pd.to_datetime(daily_city_df['date']).dt.date
        daily_city_df['date'] = pd.to_datetime(daily_city_df['date'])
        daily_city_df['city'] = city_name
        weather_dfs.append(daily_city_df)
    
    daily_df = pd.concat(weather_dfs, ignore_index=True)
    daily_df

Coordinates 48.75°N 2.25°E
Elevation 46.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 52.5°N -1.75°E
Elevation 108.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [31]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         14 non-null     datetime64[ns]
 1   temperature_2m_mean          14 non-null     float32       
 2   precipitation_sum            14 non-null     float32       
 3   wind_speed_10m_max           14 non-null     float32       
 4   wind_direction_10m_dominant  14 non-null     float32       
 5   city                         14 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 580.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [32]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-18 00:12:21,357 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1267871/fs/1262659/fg/1638064
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1267871/fs/1262659/fg/1638064


Uploading Dataframe: 100.00% |██████████| Rows 4/4 | Elapsed Time: 00:01 | Remaining Time: 00:00



Launching job: air_quality_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1267871/jobs/named/air_quality_2_offline_fg_materialization/executions
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1267871/jobs/named/air_quality_2_offline_fg_materialization/executions


(Job('air_quality_2_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 735348
         }
       },
       "result": {
         "observed_value": 42.0,
         "element_count": 4,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-17T11:12:21.000356Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectat

In [33]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-18 00:12:35,593 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1267871/fs/1262659/fg/1595979
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1267871/fs/1262659/fg/1595979


Uploading Dataframe: 100.00% |██████████| Rows 14/14 | Elapsed Time: 00:01 | Remaining Time: 00:00



Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1267871/jobs/named/weather_1_offline_fg_materialization/executions
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1267871/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-18 00:12:55,186 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-18 00:12:55,186 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-18 00:13:04,864 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 00:13:04,864 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 00:15:01,314 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-18 00:15:01,314 INFO: Waiting for execution to finish. C

(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 733221
         }
       },
       "result": {
         "observed_value": 5.351784706115723,
         "element_count": 14,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-17T11:12:35.000593Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type":

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
